In [1]:
# Importing the required libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error,  r2_score
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import kerastuner as kt
from tensorflow import keras
from tensorflow.keras import layers


In [2]:
# Loading the dataset
# Train Data
temp_df = pd.read_csv("../../data/cleaned/train.csv", nrows=0)  # Read only the header
total_columns = len(temp_df.columns)
columns_to_use = temp_df.columns[1:total_columns]
train_data = pd.read_csv("../../data/cleaned/train.csv",  usecols=columns_to_use)
print(train_data.head(5))
# Test Data
test_data = pd.read_csv("../../data/cleaned/test.csv", usecols=columns_to_use)
print(test_data.head())

            building_name        meter        date  meter_reading site_name  \
0  Bear_education_Alfredo  electricity  2016-01-01         2.9050      Bear   
1  Bear_education_Alfredo  electricity  2016-01-02         2.7700      Bear   
2  Bear_education_Alfredo  electricity  2016-01-03         2.6725      Bear   
3  Bear_education_Alfredo  electricity  2016-01-04         4.5650      Bear   
4  Bear_education_Alfredo  electricity  2016-01-05         4.7825      Bear   

  sub_primaryspaceusage    sqm    sqft    timezone  airTemperature  \
0             Education  609.8  6564.0  US/Pacific        5.246861   
1             Education  609.8  6564.0  US/Pacific        5.993973   
2             Education  609.8  6564.0  US/Pacific        5.660314   
3             Education  609.8  6564.0  US/Pacific        5.048507   
4             Education  609.8  6564.0  US/Pacific        4.745567   

   cloudCoverage  dewTemperature  precipDepth1HR  precipDepth6HR  \
0       1.927009        0.254484    

In [3]:
test_data['meter'].unique()

array(['electricity', 'chilledwater', 'gas', 'hotwater', 'solar', 'water',
       'steam', 'irrigation'], dtype=object)

In [4]:
# Dropping the columns that are not relevant to our analysis
train_data = train_data.drop(columns=['building_name', 'site_name'])
test_data = test_data.drop(columns=['building_name', 'site_name'])

train_data = train_data[(train_data['meter'] == 'electricity') | (train_data['meter'] == 'chilledwater') | (train_data['meter'] == 'steam')
| (train_data['meter'] == 'hotwater') | (train_data['meter'] == 'gas')]

test_data = test_data[(test_data['meter'] == 'electricity') | (test_data['meter'] == 'chilledwater') | (test_data['meter'] == 'steam')
| (test_data['meter'] == 'hotwater') | (test_data['meter'] == 'gas')]

# Building index on building_id for further assessment
train_data.set_index('building_id', inplace=True)
test_data.set_index('building_id', inplace=True)

In [5]:
# Filtering for gas meter_reading
train_data = train_data[train_data['meter'] == 'gas']
test_data = test_data[test_data['meter'] == 'gas']

train_data = train_data.drop(columns=['meter'])
test_data = test_data.drop(columns=['meter'])

In [6]:
# Inspecting the data frames
print(train_data.sample(2))
print('-------------------------------------------------------------')
print(test_data.sample(2))

                   date  meter_reading        sub_primaryspaceusage     sqm  \
building_id                                                                   
337          2016-05-02        377.491  Primary/Secondary Classroom  9005.0   
354          2016-09-20        211.569  Primary/Secondary Classroom  2335.0   

                sqft       timezone  airTemperature  cloudCoverage  \
building_id                                                          
337          96929.0  Europe/London       14.796923       2.082518   
354          25134.0  Europe/London       20.850439       2.058763   

             dewTemperature  precipDepth1HR  precipDepth6HR  seaLvlPressure  \
building_id                                                                   
337                9.096044        2.127750       12.484141     1016.285599   
354               14.417105        0.605524       11.645330     1017.070180   

             windDirection  windSpeed  season  site_id  
building_id                 

In [7]:
# Separating into X and Y dataframes
X_train = train_data.drop(columns=['meter_reading'])  # Exclude target variable
y_train = train_data['meter_reading']

X_test = test_data.drop(columns=['meter_reading'])  # Exclude target variable
y_test = test_data['meter_reading']

In [8]:
# Convert 'site_id' from numeric to categorical
X_train['site_id'] = X_train['site_id'].astype('category')
X_test['site_id'] = X_test['site_id'].astype('category')

In [9]:
print(X_train.dtypes)
print(X_train.columns)

date                       object
sub_primaryspaceusage      object
sqm                       float64
sqft                      float64
timezone                   object
airTemperature            float64
cloudCoverage             float64
dewTemperature            float64
precipDepth1HR            float64
precipDepth6HR            float64
seaLvlPressure            float64
windDirection             float64
windSpeed                 float64
season                     object
site_id                  category
dtype: object
Index(['date', 'sub_primaryspaceusage', 'sqm', 'sqft', 'timezone',
       'airTemperature', 'cloudCoverage', 'dewTemperature', 'precipDepth1HR',
       'precipDepth6HR', 'seaLvlPressure', 'windDirection', 'windSpeed',
       'season', 'site_id'],
      dtype='object')


In [10]:
# Define features and types based on your dataset
numerical_features = ['sqm', 'sqft', 'airTemperature', 'cloudCoverage', 'dewTemperature',
                      'precipDepth1HR', 'precipDepth6HR', 'seaLvlPressure', 'windDirection', 'windSpeed']
categorical_features = ['timezone', 'season', 'sub_primaryspaceusage', 'site_id']

In [11]:
# Create a preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

In [12]:
# Create a preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Fit the preprocessor on the training data and transform both training and test data
preprocessor.fit(X_train)
X_train_processed = preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(X_test)

In [13]:
# Convert the processed data back to dense DataFrames
X_train_processed_df = pd.DataFrame(X_train_processed, columns=preprocessor.get_feature_names_out())
X_test_processed_df = pd.DataFrame(X_test_processed, columns=preprocessor.get_feature_names_out())

In [14]:
# Checking the columns
X_train_processed_df.columns

Index(['num__sqm', 'num__sqft', 'num__airTemperature', 'num__cloudCoverage',
       'num__dewTemperature', 'num__precipDepth1HR', 'num__precipDepth6HR',
       'num__seaLvlPressure', 'num__windDirection', 'num__windSpeed',
       'cat__timezone_Europe/Dublin', 'cat__timezone_Europe/London',
       'cat__timezone_US/Eastern', 'cat__timezone_US/Mountain',
       'cat__season_Fall', 'cat__season_Spring', 'cat__season_Summer',
       'cat__season_Winter', 'cat__sub_primaryspaceusage_Academic',
       'cat__sub_primaryspaceusage_Classroom',
       'cat__sub_primaryspaceusage_College Classroom',
       'cat__sub_primaryspaceusage_Primary/Secondary Classroom',
       'cat__sub_primaryspaceusage_Research', 'cat__site_id_2',
       'cat__site_id_9', 'cat__site_id_11', 'cat__site_id_15'],
      dtype='object')

In [15]:
y_train_scaled = np.log1p(y_train.values.reshape(-1, 1))
y_test_scaled = np.log1p(y_test.values.reshape(-1, 1))

### Neural Networks

In [16]:
# Define the neural network model
model = Sequential()
model.add(Dense(64, input_dim=X_train_processed_df.shape[1], activation='relu')) # Adjust input_dim based on your features
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))  # Output layer, adjust units and activation based on your output

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

In [17]:
# Fit the model to the training data
model.fit(X_train_processed, y_train_scaled, epochs=10, batch_size=10, verbose=1)

# Predict the values for X_train
y_pred = model.predict(X_train_processed_df)

Epoch 1/10
3660/3660 [==============================] - 1s 306us/step - loss: 9.1763
Epoch 2/10
3660/3660 [==============================] - 1s 303us/step - loss: 8.4047
Epoch 3/10
3660/3660 [==============================] - 1s 295us/step - loss: 7.8526
Epoch 4/10
3660/3660 [==============================] - 1s 296us/step - loss: 7.6339
Epoch 5/10
3660/3660 [==============================] - 1s 300us/step - loss: 7.5202
Epoch 6/10
3660/3660 [==============================] - 1s 295us/step - loss: 7.3880
Epoch 7/10
3660/3660 [==============================] - 1s 299us/step - loss: 7.3210
Epoch 8/10
3660/3660 [==============================] - 1s 295us/step - loss: 7.2094
Epoch 9/10
3660/3660 [==============================] - 1s 295us/step - loss: 7.1392
Epoch 10/10
1144/1144 [==============================] - 0s 212us/step


In [18]:
#mae = mean_absolute_error(y_train_scaled, y_pred)
mse = mean_squared_error(y_train_scaled, y_pred)
r2 = r2_score(y_train_scaled, y_pred)

#print('Mean Absolute Error:', mae)
print('Mean Squared Error:', mse)
print('R-squared Score:', r2)


Mean Squared Error: 6.920020025917805
R-squared Score: 0.4397869655566857


In [19]:
# Predict the values for X_test
y_pred = model.predict(X_test_processed_df)
y_pred = np.nan_to_num(y_pred, nan=0)
mse = mean_squared_error(y_test_scaled, y_pred)
r2 = r2_score(y_test_scaled, y_pred)
print('Mean Squared Error:', mse)
print('R-squared Score:', r2)

1141/1141 [==============================] - 0s 212us/step
Mean Squared Error: 12.053200983873037
R-squared Score: 0.17413665786437782


In [20]:

def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Dense(
        units=hp.Int('input_units', min_value=32, max_value=256, step=64),  # Reduced max value and increased step
        activation='relu',
        input_shape=(X_train_processed.shape[1],)
    ))

    # Reduced the maximum number of layers
    for i in range(hp.Int('n_layers', 1, 3)):
        model.add(layers.Dense(
            units=hp.Int(f'layer_{i}_units', min_value=32, max_value=256, step=64),  # Simplify layers
            activation='relu'
        ))

    model.add(layers.Dense(1)) 

    model.compile(
        optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-3, 1e-4])),  # Simplified choices
        loss='mse',  
        metrics=['mae', 'mse'] 
    )

    return model

tuner = kt.RandomSearch(
    build_model,
    objective='val_mse', 
    max_trials=10,  
    executions_per_trial=2,  
    directory='my_dir',
    project_name='quick_tune'
)

tuner.search(
    X_train_processed_df, y_train_scaled,
    epochs=5,  # Reduced number of epochs for tuning
    validation_split=0.1,  # Adjust based on your dataset
    batch_size=256  # Increased batch size for faster processing
)

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the model with the best hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
history = model.fit(
    X_train_processed_df, y_train_scaled,
    epochs=10,  # You can increase this for the final model training
    validation_split=0.1,  # Keep consistent with tuning phase or adjust as necessary
    batch_size=256  # Set a fixed batch size
)

Reloading Tuner from my_dir/quick_tune/tuner0.json


/var/folders/8x/wqmfhzdj59vcq2kqnk5_9pp40000gn/T/ipykernel_68860/214843204.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


Epoch 1/10
129/129 [==============================] - 0s 1ms/step - loss: 27.4214 - mae: 4.4356 - mse: 27.4214 - val_loss: 54.0271 - val_mae: 6.8519 - val_mse: 54.0271
Epoch 2/10
129/129 [==============================] - 0s 875us/step - loss: 12.2223 - mae: 2.9292 - mse: 12.2223 - val_loss: 31.5023 - val_mae: 5.4046 - val_mse: 31.5023
Epoch 3/10
129/129 [==============================] - 0s 837us/step - loss: 9.4755 - mae: 2.5074 - mse: 9.4755 - val_loss: 27.6213 - val_mae: 5.0656 - val_mse: 27.6213
Epoch 4/10
129/129 [==============================] - 0s 855us/step - loss: 8.6373 - mae: 2.3787 - mse: 8.6373 - val_loss: 25.7506 - val_mae: 4.8789 - val_mse: 25.7506
Epoch 5/10
129/129 [==============================] - 0s 853us/step - loss: 8.2255 - mae: 2.3170 - mse: 8.2255 - val_loss: 25.0196 - val_mae: 4.8001 - val_mse: 25.0196
Epoch 6/10
129/129 [==============================] - 0s 837us/step - loss: 8.0009 - mae: 2.2733 - mse: 8.0009 - val_loss: 25.1547 - val_mae: 4.8150 - val_mse

In [21]:
# Check and replace NaN values in the test features 
if X_test_processed_df.isna().any().any(): 
    X_test_processed_df = X_test_processed_df.fillna(0)  # Replace NaN with 0 

# Check and replace NaN values in the test labels 
if np.isnan(y_test_scaled).any():  
    y_test_scaled = np.nan_to_num(y_test_scaled)  # Replace NaN with 0 (default)

In [22]:
test_loss, test_mae, test_mse = model.evaluate(X_test_processed_df, y_test_scaled, batch_size=256)
print(f'Test Loss: {test_loss}, Test MAE: {test_mae}, Test MSE: {test_mse}')

143/143 [==============================] - 0s 369us/step - loss: 12.1281 - mae: 2.8839 - mse: 12.1281
Test Loss: 12.12814712524414, Test MAE: 2.8838841915130615, Test MSE: 12.12814712524414


In [23]:
y_pred = model.predict(X_test_processed_df, batch_size=256)

# Calculate R-squared score
r_squared = r2_score(y_test_scaled, y_pred)
print(f'R-squared Score: {r_squared}')

143/143 [==============================] - 0s 333us/step
R-squared Score: 0.16900147187044778
